<a href="https://colab.research.google.com/github/AyushiKashyapp/KnowledgeGraphs/blob/main/LNNComputation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computation with **Logical Neural Network**

LNN package from IBM allows to define aa neurosymbolic model.

In [1]:
%%capture
!pip install git+https://github.com/IBM/LNN

In [2]:
from lnn import Model

model = Model()

We can create the predicates as:

smokes(X)

cancer(Y)

friends(X, Y)

In [4]:
from lnn import Predicates, World

Smokes = Predicates('Smokes')
Cancer = Predicates('Cancer', world = World.CLOSED)
Friends = Predicates('Friends', arity = 2) # 2 arguments.

Variable and Axioms

In [5]:
from lnn import Variables
from lnn import Implies, Iff

x, y = Variables('x', 'y')

Smoking_causes_Cancer = Implies(Smokes(x), Cancer(x))

Smokers_befriend_Smokers = Implies(Friends(x, y), Iff(Smokes(x), Smokes(y)))

These axioms can be added to the model under open or closed world assumptions.

In [6]:
from lnn import World

formulae = [
    Smoking_causes_Cancer,
    Smokers_befriend_Smokers
]
model.add_knowledge(*formulae, world = World.AXIOM)

Adding data to the model.

In [7]:
from lnn import Fact

model.add_data({
    Friends: {
        ('Anna', 'Bob'): Fact.TRUE,
        ('Bob', 'Anna'): Fact.TRUE,
        ('Anna', 'Edward'): Fact.TRUE,
        ('Edward', 'Anna'): Fact.TRUE,
        ('Anna', 'Frank'): Fact.TRUE,
        ('Frank', 'Anna'): Fact.TRUE,
        ('Bob', 'Chris'): Fact.TRUE},
    Smokes: {
        'Anna': Fact.TRUE,
        'Edward': Fact.TRUE,
        'Frank': Fact.TRUE,
        'Gary': Fact.TRUE},
    Cancer: {
        'Anna': Fact.TRUE,
        'Edward': Fact.TRUE}
    })
model.print()


***************************************************************************
                                LNN Model

AXIOM Implies: (Friends(0, 1) → ((Smokes(0) → Smokes(1)) ∧ (Smokes(1) → Smokes(0)))) 

OPEN Iff: ((Smokes(0) → Smokes(1)) ∧ (Smokes(1) → Smokes(0))) 

OPEN Implies: (Smokes(0) → Smokes(1)) 

OPEN Predicate: Friends 
('Bob', 'Chris')                                            TRUE (1.0, 1.0)
('Edward', 'Anna')                                          TRUE (1.0, 1.0)
('Anna', 'Frank')                                           TRUE (1.0, 1.0)
('Frank', 'Anna')                                           TRUE (1.0, 1.0)
('Bob', 'Anna')                                             TRUE (1.0, 1.0)
('Anna', 'Edward')                                          TRUE (1.0, 1.0)
('Anna', 'Bob')                                             TRUE (1.0, 1.0)

AXIOM Implies: (Smokes(0) → Cancer(0)) 

CLOSED Predicate: Cancer 
('Edward',)                                                 TRUE

In [8]:
model.infer()
model.print()


***************************************************************************
                                LNN Model

AXIOM Implies: (Friends(0, 1) → ((Smokes(0) → Smokes(1)) ∧ (Smokes(1) → Smokes(0)))) 
('Bob', 'Chris')                                            TRUE (1.0, 1.0)
('Anna', 'Anna')                                            TRUE (1.0, 1.0)
('Frank', 'Anna')                                           TRUE (1.0, 1.0)
('Anna', 'Frank')                                           TRUE (1.0, 1.0)
('Bob', 'Anna')                                             TRUE (1.0, 1.0)
('Gary', 'Frank')                                           TRUE (1.0, 1.0)
('Edward', 'Gary')                                          TRUE (1.0, 1.0)
('Frank', 'Frank')                                          TRUE (1.0, 1.0)
('Gary', 'Anna')                                            TRUE (1.0, 1.0)
('Anna', 'Bob')                                             TRUE (1.0, 1.0)
('Edward', 'Anna')                

# DeepProbLog

DeepProbLog is another framework for neurosymbolic reasoning, it can be used to learn sums of digits from the MNIST dataset.

In [9]:
%%capture
!pip install deepproblog
!wget https://github.com/ML-KULeuven/deepproblog/raw/master/src/deepproblog/examples/minimal/addition.pl

Loading the MNIST dataset

In [10]:
from typing import Mapping, Iterator

import torch
import torchvision
import torchvision.transforms as transforms
from problog.logic import Term, Constant

from deepproblog.dataset import Dataset
from deepproblog.query import Query

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)

datasets = {
    "train": torchvision.datasets.MNIST(
        root='data/', train=True, download=True, transform=transform
    ),
    "test": torchvision.datasets.MNIST(
        root='data/', train=False, download=True, transform=transform
    ),
}


class MNISTImages(Mapping[Term, torch.Tensor]):

    def __iter__(self) -> Iterator:
        for i in range(self.dataset):
            yield self.dataset[i][0]

    def __len__(self) -> int:
        return len(self.dataset)

    def __init__(self, subset):
        self.subset = subset
        self.dataset = datasets[self.subset]

    def __getitem__(self, item):
        return self.dataset[int(item[0])][0]


class AdditionDataset(Dataset):

    def __init__(self, subset):
        self.subset = subset
        self.dataset = datasets[subset]

    def __len__(self):
        return len(self.dataset) // 2

    def to_query(self, i: int) -> Query:
        image1 = Term("tensor", Term(self.subset, Constant(i * 2)))
        image2 = Term("tensor", Term(self.subset, Constant(i * 2 + 1)))
        label = Constant(int(self.dataset[i*2][1] + self.dataset[i*2+1][1]))
        term = Term('addition', image1, image2, label)
        return Query(term)

dataset = AdditionDataset("train")

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 35760207.50it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1121170.02it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 10074293.10it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3332842.68it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



Converting MNIST data to images.

In [11]:
import torchvision.transforms as T
from PIL import Image

transform = T.ToPILImage()
transform(dataset.dataset[2][0])

In [12]:
transform(dataset.dataset[3][0])

In [13]:
dataset.to_query(1)

(1.0::addition(tensor(train(2)),tensor(train(3)),5), {})

Defining a neural network to solve MNIST

In [14]:
import torch.nn as nn


class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 6, 5),
            nn.MaxPool2d(2, 2),  # 6 24 24 -> 6 12 12
            nn.ReLU(True),
            nn.Conv2d(6, 16, 5),  # 6 12 12 -> 16 8 8
            nn.MaxPool2d(2, 2),  # 16 8 8 -> 16 4 4
            nn.ReLU(True),
        )
        self.classifier = nn.Sequential(
            nn.Linear(16 * 4 * 4, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10),
            nn.Softmax(1),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = x.view(-1, 16 * 4 * 4)
        x = self.classifier(x)
        return x

Training the model

In [15]:
import torch

from deepproblog.dataset import DataLoader
from deepproblog.engines import ExactEngine
from deepproblog.model import Model
from deepproblog.network import Network
from deepproblog.train import train_model

network = MNIST_Net()
net = Network(network, "mnist_net", batching=True)
net.optimizer = torch.optim.Adam(network.parameters(), lr=1e-3)

model = Model("addition.pl", [net])
model.set_engine(ExactEngine(model))
model.add_tensor_source("train", MNISTImages("train"))
model.add_tensor_source("test", MNISTImages("test"))

dataset = AdditionDataset("train")

# Train the model
loader = DataLoader(dataset, 2, False)
train_model(model, loader, 1, log_iter=100, profile=0)
model.save_state("snapshot/trained_model.pth")

# Query the model
query = dataset.to_query(0)
print(query)
result = model.solve([query])[0]
print(result)

/usr/local/lib/python3.10/dist-packages/deepproblog/engines/__init__.py:6: UserWarning: ApproximateEngine is not available as PySwip could not be found
  warnings.warn("ApproximateEngine is not available as PySwip could not be found")


Training  for 1 epoch(s)
Epoch 1
Iteration:  100 	s:12.3677 	Average Loss:  2.762265934944153
Iteration:  200 	s:12.9491 	Average Loss:  2.7005186969041826
Iteration:  300 	s:12.3854 	Average Loss:  2.4862266620993614
Iteration:  400 	s:13.6648 	Average Loss:  2.0800369179248808
Iteration:  500 	s:12.9458 	Average Loss:  1.2904680143034784
Iteration:  600 	s:13.7440 	Average Loss:  0.8459320670925081
Iteration:  700 	s:12.1735 	Average Loss:  0.9239749544451479
Iteration:  800 	s:12.8871 	Average Loss:  0.5996868947453913
Iteration:  900 	s:13.0257 	Average Loss:  0.59107800071768
Iteration:  1000 	s:13.0264 	Average Loss:  0.47710468759207286
Iteration:  1100 	s:12.0098 	Average Loss:  0.5075771557187545
Iteration:  1200 	s:11.9928 	Average Loss:  0.459920847316389
Iteration:  1300 	s:12.7151 	Average Loss:  0.3836900119949678
Iteration:  1400 	s:11.9159 	Average Loss:  0.41574702696923355
Iteration:  1500 	s:12.1045 	Average Loss:  0.39746683034129093
Iteration:  1600 	s:12.5280 	Ave